In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_pickle("/kaggle/input/data0-25/data_norm0.25.pkl")
df2 = pd.read_pickle("/kaggle/input/data1-0/data_norm1.pkl")
df3 = pd.read_pickle("/kaggle/input/data0-5/data_norm0.5.pkl")

In [3]:
df = pd.concat([df1, df2])
df = pd.concat([df, df3])

In [4]:
df = df.reset_index() 

In [5]:
df = df.drop("index",  axis=1)

In [6]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=127)

In [7]:
del df1
del df2
del df3

In [8]:
df

,OUTPUT,INPUT
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.1242904446...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0770836309982...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0.10748772366243657, 0, 0, 0.0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0.06577925136276556, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
1292574,"[0, 0, 0.23119837918566855, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1292575,"[0, 0, 0, 0, 0, 0, 0, 0.07482285803921773, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1292576,"[0, 0, 0.13130826532079368, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1292577,"[0.0866654566110813, 0.0684285664685669, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
df_test = df_test.reset_index() 
df_train = df_train.reset_index() 

In [10]:
df_train = df_train.drop("index",  axis=1)
df_test = df_test.drop("index",  axis=1)

In [13]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.tensor(self.data.iloc[idx, 0], dtype=torch.float)
        y = torch.tensor(self.data.iloc[idx, 1], dtype=torch.float)
        return x, y

 
datatest = CustomDataset(df_test) 
dataloader_test = DataLoader(datatest, batch_size=32, shuffle=False)

evaluate

0.00000011920928955078125

In [14]:
models_list = ["/kaggle/input/version-4.0/pytorch/v4.0/2/989_x3_49_epoch1.pth", 
              "/kaggle/input/version-4.0/pytorch/v4.0/2/989_x3_49_epoch2.pth", 
              "/kaggle/input/version-4.0/pytorch/v4.0/2/989_x3_49_epoch3.pth", 
              "/kaggle/input/version-4.0/pytorch/v4.0/2/989_x3_49_epoch4.pth",
              "/kaggle/input/version-4.0/pytorch/v4.0/2/989_x3_49_epoch5.pth", 
              "/kaggle/input/version-4.0/pytorch/v4.0/2/989_x3_49_epoch6.pth"]

In [15]:
import numpy
from sklearn.metrics import precision_score, recall_score


for i in models_list:
    model = torch.load(i)
    model.eval()
    recall_list = []
    f1_list = []
    precision_list = []
    accuracy_list = []
    for targets, inputs in dataloader_test:
        out = model.forward(inputs).detach().numpy()
        targets = targets.detach().numpy()
        out = np.argmax(out, axis=1)
        targets = np.argmax(targets, axis=1)
        threshold = 0.1
        out = (out >= threshold).astype(int)
        targets = (targets >= threshold).astype(int)


        precision = precision_score(targets, out, average=None, zero_division=1)
        recall = recall_score(targets, out, average=None, zero_division=1)
    
        precision_list.extend(precision)
        recall_list.extend(recall)

    
    print(i.split("/")[-1])
    print("recall:", sum(recall_list)/len(recall_list))
    print("precision:", sum(precision)/len(precision))
    print("\n")

989_x3_49_epoch1.pth
recall: 0.9731345809696823
precision: 0.75


989_x3_49_epoch2.pth
recall: 0.9748768172806537
precision: 0.75


989_x3_49_epoch3.pth
recall: 0.9581160026175181
precision: 0.75


989_x3_49_epoch4.pth
recall: 0.9531329965131549
precision: 1.0


989_x3_49_epoch5.pth
recall: 0.9341635240357548
precision: 1.0


989_x3_49_epoch6.pth
recall: 0.9480118798656033
precision: 0.75


